In [ ]:
import os
import numpy as np
import pandas as pd
import time
import copy
import datetime
from datetime import datetime

# 忽略警告訊息
import warnings
warnings.filterwarnings( 'ignore' )

### 輸入相關資訊

In [ ]:
""" 輸入評估日 """
valuation_date = '2018-03-31'

""" 選擇佣金費用處理方式 """
# amortization : 分期攤銷；Expensing：費用化
Method = 'amortization'

""" 輸入資料儲存路徑 """
dir_data = '/Users/YenLin/Desktop/PAA/Raw_Data/'

""" 輸入當期保單及保費資料檔名 """
policy_data_name = '保單資料_2018Q1.txt'
premium_data_name = '保費資料_2018Q1.txt'
loss_data_name = '賠款資料_2018Q1.txt'

# 評估日所屬年度及季別
valuation_year = int( valuation_date[ 0:4 ] )
if ( valuation_date[ 5:7 ] == '03' ) : 
    Quarter = 'Q1'
    previous_Quarter = 'Q4'
    previous_valuation_year = valuation_year - 1 
elif ( valuation_date[ 5:7 ] == '06' ) : 
    Quarter = 'Q2'
    previous_Quarter = 'Q1'
    previous_valuation_year = valuation_year 
elif ( valuation_date[ 5:7 ] == '09' ) : 
    Quarter = 'Q3'  
    previous_Quarter = 'Q2'
    previous_valuation_year = valuation_year 
else : 
    Quarter = 'Q4'  
    previous_Quarter = 'Q3'
    previous_valuation_year = valuation_year 


### 讀取資料

In [ ]:
policy_data = os.path.join( dir_data, policy_data_name )
premium_data = os.path.join( dir_data, premium_data_name )
loss_data = os.path.join( dir_data, loss_data_name )

print( 'Path of Policy Data: %s' % ( policy_data ) )
print( 'Path of Premium Data: %s' % ( premium_data ) )
print( 'Path of loss Data: %s' % ( loss_data ) )

# 資料讀取
Policy_data = pd.read_csv( policy_data, encoding = 'utf-8' )
Premium_data = pd.read_csv( premium_data, encoding = 'utf-8' )
Loss_data = pd.read_csv( loss_data, encoding = 'utf-8' )

### 資料格式調整
- 日期欄位  
- 百分比轉浮點小數

In [ ]:
Policy_date_col = [ '簽單日', '保費到期日', '保險起日', '保險迄日' ]
Policy_percentage_col = [ '佣金率' ]
Premium_date_col = [ '保費收取日', '佣金支付日' ]
Loss_date_col = [ '保險起日', '保險迄日', '出險日' ]

for col in Policy_date_col :
    Policy_data[ col ] = Policy_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 

for col in Policy_percentage_col :  
    Policy_data[ col ] = Policy_data[ col ].str.strip( '%' ).astype( float ) / 100 

for col in Premium_date_col :
    Premium_data[ col ] = Premium_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 

for col in Loss_date_col :
    Loss_data[ col ] = Loss_data[ col ].apply( lambda X : datetime.strptime( X, '%Y-%m-%d' ).date( ) ) 


### 檢視當期資料（ 前15筆 ）

In [ ]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保單資料筆數 = ' + str( Policy_data.shape[0] ) + '\n' )
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保單資料 : ' ) 
Policy_data.head( 15 )

In [ ]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保費資料筆數 = ' + str( Premium_data.shape[0] ) + '\n' )
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季保費資料 : ' ) 
Premium_data.head( 15 )

In [ ]:
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季賠款資料筆數 = ' + str( Loss_data.shape[0] ) + '\n' )
print( str( valuation_year ) + ' ' + str( Quarter ) + '  當季賠款資料 : ' ) 
Loss_data.head( 15 )

### 合併當季的保單及保費資料

In [ ]:
cols_name = [ '保批單號碼', 'Group', '簽單日', '保費到期日', '保險起日', '保險迄日', '簽單保費', '佣金率', '保費收取日', '保費收取金額', '佣金支付日', '佣金支付金額' ]

current_data = pd.merge( Policy_data, Premium_data, on = [ '保批單號碼' ], how = 'outer' )
current_data = current_data[ cols_name ]                 # 調整欄位順序
current_data_temp = copy.deepcopy( current_data )  # 複製資料

print( '檢視合併後資料：' )
current_data.head( 15 )

### 計算「 初始認列日 」
保險合約群組(group)應於下列日期中最早者認列：
- 該群組保障期間開始日
- 該群組之被保險人第一期保費到期日
- 對於虧損性合約群組，當該群組已成為虧損時點

In [ ]:
current_data[ '初始認列日' ] = np.nan

# 選取合約群組中最早認列的日期
earliest_date = current_data[ [ '保費到期日', '保險起日' ] ].min( 1 ).min( )

# 判斷群組中每張保單的「初始認列日」
for i in np.arange( current_data.shape[0] ) :
    if min( current_data[ '保費到期日' ][ i ], current_data[ '保險起日' ][ i ] ) == earliest_date :
        current_data[ '初始認列日' ][ i ] = earliest_date
    elif  min( current_data[ '保費到期日' ][ i ], current_data[ '保險起日' ][ i ] ) > earliest_date :
        current_data[ '初始認列日' ][ i ] = min( current_data[ '簽單日' ][ i ], current_data[ '保費到期日' ][ i ], current_data[ '保險起日' ][ i ] ) 
    else :
        current_data[ '初始認列日' ][ i ] = np.nan

        
current_data

### 計算每張保單的「 保期(月) 」及「 當期已過月數 」

In [ ]:
# 定義函數，用以計算月份數
def months( data, evaluate_date ) :
    start_year = data[ '保險起日' ].apply( lambda X : X.year )                             # 年度 of 保險起日
    end_year = data[ '保險迄日' ].apply( lambda X : X.year )                              # 年度 of 保險迄日
    start_month = data[ '保險起日' ].apply( lambda X : X.month )                       # 月份 of 保險起日
    end_month = data[ '保險迄日' ].apply( lambda X : X.month )                        # 月份 of 保險迄日
    evaluate_date = datetime.strptime( evaluate_date, '%Y-%m-%d' ).date( )    # 評估日
    evaluate_year = evaluate_date.year                                                          # 評估日所屬年度
    evaluate_month = evaluate_date.month                                                    # 評估日所屬月份
    
    if ( evaluate_month == 3 ) : 
        q_start_date = str( evaluate_year ) + '-' + '01-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )   # 評估季的第一天日期   
    elif ( evaluate_month == 6 ) :
        q_start_date = str( evaluate_year ) + '-' + '04-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )   # 評估季的第一天日期
    elif ( evaluate_month == 9 ) :
        q_start_date = str( evaluate_year ) + '-' + '07-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  # 評估季的第一天日期
    else :  
        q_start_date = str( evaluate_year ) + '-' + '10-01'
        q_start_date =  datetime.strptime( q_start_date, '%Y-%m-%d' ).date( )  # 評估季的第一天日期

    # 計算『保期(月)』
    data[ '保期(月)' ] = ( end_year - start_year )*12 + ( end_month - start_month + 1 )
    
    # 計算『當期已經過月數』
    data[ '當期已經過月數' ] = np.nan
    for i in np.arange( data.shape[0] ) : 
        first_date = max( q_start_date,  data[ '保險起日' ][ i ] )
        least_date = min( evaluate_date, data[ '保險迄日' ][ i ] ) 
        first_year = first_date.year
        first_month = first_date.month
        least_year = least_date.year
        least_month = least_date.month
        data[ '當期已經過月數' ][ i ] = ( least_year - first_year )*12 + ( least_month - first_month + 1 )

    return data.head( 15 )


months( current_data, valuation_date )

### 計算及填補下列項目：
- (1) 計算「應付佣金」
- (2) 填補「期初LRC」
- (3) 對於尚未收取保費或支付佣金的保單，將「保費收取金額」及「佣金支付金額」 補 0 
- (4) 填補「當期已收保費」（ = 保費收取金額 ）
- (5) 填補「當期已付佣金」（ = 佣金支付金額 ）
- (6) 計算「當期認列之合約收入」  
$ \displaystyle 當期認列之合約收入= 簽單保費  \times \frac{當期已經過月數}{保期(月)} $  
#####   
- (7) 計算「取得成本攤銷」  
$ \displaystyle 取得成本攤銷 = 簽單保費  \times 佣金率 \times \frac{當期已經過月數}{保期(月)} $  
#####  
- (8) 計算「期末LRC」  
( a ) 佣金分期攤銷( amortization )：  
$ 期末LRC = 期初LRC + 當期已收保費 - 當期已付佣金 - 當期認列之合約收入 + 取得成本攤銷 $   
( b ) 佣金費用化：  
$ 期末LRC = 期初LRC + 當期已收保費 - 當期認列之合約收入 $   
#####   

- (9) 計算「累積已收保費」
- (10) 計算「累積已付佣金」

In [ ]:
# 計算「應付佣金」
current_data[ '應付佣金' ] = current_data.apply( lambda X : X[ '簽單保費' ] * X[ '佣金率' ], axis = 1 )

# 填補「期初LRC」
current_data[ '期初LRC' ] = 0

# 填補「當期已收保費」及「當期已付佣金」
current_data = current_data.fillna( { '保費收取金額' : 0, '佣金支付金額' : 0 } )   
current_data[ '當期已收保費' ] = current_data[ '保費收取金額' ]
current_data[ '當期已付佣金' ] = current_data[ '佣金支付金額' ]

# 計算「當期認列之合約收入」
current_data[ '當期認列之合約收入' ] = current_data.apply( lambda X : X[ '簽單保費' ] * ( X[ '當期已經過月數' ] / X[ '保期(月)' ] ), axis = 1 )

# 計算「取得成本攤銷」
current_data[ '取得成本攤銷' ] = current_data.apply( lambda X : X[ '簽單保費' ] * X[ '佣金率' ] * ( X[ '當期已經過月數' ] / X[ '保期(月)' ] ), axis = 1 )

# 計算「期末LRC」
current_data[ '期末LRC(amortization)' ] = current_data.apply( lambda X : X[ '期初LRC' ] + X[ '當期已收保費' ] - X[ '當期已付佣金' ] - X[ '當期認列之合約收入' ] + X[ '取得成本攤銷' ], axis=1 )
current_data[ '期末LRC(expensing)' ] = current_data.apply( lambda X : X[ '期初LRC' ] + X[ '當期已收保費' ] - X[ '當期認列之合約收入' ], axis=1 )

# 計算「累積已收保費」及「累積已付佣金」
current_data[ '累積已收保費' ] = current_data[ '保費收取金額' ]
current_data[ '累積已付佣金' ] = current_data[ '佣金支付金額' ]


current_data

### 「 財務狀況表」及「 財務績效表（當季）」

In [ ]:
item = str( valuation_year ) + '_' + Quarter 
previous_item = str( previous_valuation_year ) + ' ' + previous_Quarter 

select_col = [ '保批單號碼', '應付佣金', '期初LRC', '當期已收保費', '當期已付佣金', '當期認列之合約收入', '取得成本攤銷', '期末LRC(amortization)', '期末LRC(expensing)', '累積已收保費', '累積已付佣金' ]
statement_data = current_data[ select_col ]
statement_data

In [ ]:
# 「 財務狀況表 」( Statement of Financial Position )

if  ( Method == 'amortization' ) :   # amortization : 分期攤銷；Expensing：費用化
    Position_table = pd.DataFrame( columns = [ '現金', '合約資產', '合約負債' ], index = [ item ]  )
    
    # 當期「 現金 」 = 前期現金( =0 ) + 當期已收保費 - 當期已付佣金 ( 尚未考慮當期賠款！！ )
    Position_table.loc[ item, '現金' ] = statement_data[ '當期已收保費' ].sum( ) - statement_data[ '當期已付佣金' ].sum( ) 
    # 「 合約資產 」或「 合約負債 」
    LRC = statement_data[ '期末LRC(amortization)' ].sum( )
    if LRC > 0 :
        Position_table.loc[ item, '合約資產' ] = 0
        Position_table.loc[ item, '合約負債' ] = LRC
    elif LRC < 0 :
        Position_table.loc[ item, '合約資產' ] = -LRC
        Position_table.loc[ item, '合約負債' ] = 0
    else : 
        Position_table.loc[ item, '合約資產' ] = 0 
        Position_table.loc[ item, '合約負債' ] = 0
    
    # 「 業主權益 」= 前期現金 + 合約資產 - 合約負債
    Position_table[ '業主權益' ] = Position_table.apply( lambda X : X[ '現金' ] + X[ '合約資產' ] - X[ '合約負債' ], axis = 1 )

else :
    Position_table = pd.DataFrame( columns = [ '現金', '合約資產', '合約負債', '應付佣金' ], index = [ item ]  )
    
    # 當期「 現金 」 = 前期現金( =0 ) + 累積已收保費 - 累積已付佣金 ( 尚未考慮當期賠款！！ )
    Position_table.loc[ item, '現金' ]  = statement_data[ '累積已收保費' ].sum( ) - statement_data[ '累積已付佣金' ].sum( ) 
    # 「 合約資產 」或「 合約負債 」
    LRC = statement_data[ '期末LRC(expensing)' ].sum( )
    if LRC > 0 :
        Position_table.loc[ item, '合約資產' ] = 0
        Position_table.loc[ item, '合約負債' ] = LRC
    elif LRC < 0 :
        Position_table.loc[ item, '合約資產' ] = -LRC
        Position_table.loc[ item, '合約負債' ] = 0
    else : 
        Position_table.loc[ item, '合約資產' ] = 0 
        Position_table.loc[ item, '合約負債' ] = 0
    
    # 「 應付佣金 」
    Position_table.loc[ item, '應付佣金' ] = statement_data[ statement_data[ '累積已收保費' ] == 0 ][ '應付佣金' ].sum( )
    # 「 業主權益 」= 當期現金 + 合約資產 - 合約負債 - 應付佣金
    Position_table[ '業主權益' ] = Position_table.apply( lambda X : X[ '現金' ] + X[ '合約資產' ] - X[ '合約負債' ] - X[ '應付佣金' ], axis = 1 )


Position_table.reset_index( inplace = True ) 
Position_table.rename( columns = { Position_table.columns[0]: "Quarter" }, inplace = True ) # 更改欄位名稱

print( str( valuation_year ) + ' ' + str( Quarter ) + '  財務狀況表( ' + str( Method ) + ' ) ：' )
Position_table

In [ ]:
# 「 財務績效表（當季）」( Statement of Financial Performance )

if ( Method == 'amortization' ) :  # amortization : 分期攤銷；Expensing：費用化
    Performance_table = pd.DataFrame( columns = [ '保險合約收入', '已發生賠款', '取得成本攤銷' ], index = [ item ]  )
    
    # 「 保險合約收入 」
    Performance_table.loc[ item, '保險合約收入' ] = statement_data[ '當期認列之合約收入' ].sum( )
    # 「 取得成本攤銷 」
    Performance_table.loc[ item, '取得成本攤銷' ] = statement_data[ '取得成本攤銷' ].sum( )
    # 「 已發生賠款 」
    Performance_table.loc[ item, '已發生賠款' ] = 0 # (尚未考慮當期賠款！！)
    # 「 保險損益 」
    Performance_table[ '保險損益' ] = Performance_table.apply( lambda X : X[ '保險合約收入' ] - X[ '已發生賠款' ] - X[ '取得成本攤銷' ], axis=1 )
    
else :
    Performance_table = pd.DataFrame( columns = [ '保險合約收入', '已發生賠款', '佣金費用',  ], index = [ item ]  )
    
    # 「 保險合約收入 」
    Performance_table.loc[ item, '保險合約收入' ] = statement_data[ '當期認列之合約收入' ].sum( )
    # 「 已發生賠款 」
    Performance_table.loc[ item, '已發生賠款' ] = 0 # (尚未考慮當期賠款！！)
    # 「 佣金費用 」
    Performance_table.loc[ item, '佣金費用' ] = statement_data[ '應付佣金' ].sum( )
    # 「 保險損益 」
    Performance_table[ '保險損益' ] = Performance_table.apply( lambda X : X[ '保險合約收入' ] - X[ '已發生賠款' ] - X[ '佣金費用' ], axis=1 )
    
    
Performance_table.reset_index( inplace = True ) 
Performance_table.rename( columns = { Performance_table.columns[0]: "Quarter" }, inplace = True ) # 更改欄位名稱

print( str( valuation_year ) + ' ' + str( Quarter ) + ' 財務績效表(當季)  ( ' + str( Method ) + ' ) ：'  )
Performance_table

### 儲存檔案 

In [ ]:
current_data_temp.to_csv( str( item ) + '_data.txt', index = False, encoding = 'utf-8' ) 
statement_data.to_csv( str( item ) + '_statement_data.txt', index = False, encoding = 'utf-8' ) 
Position_table.to_csv( str( item ) + '_Position_table.txt', index = False, encoding = 'utf-8' ) 
Performance_table.to_csv( str( item ) + '_Performance_table.txt', index = False, encoding = 'utf-8' ) 